In [15]:
import pandas as pd

In [34]:
df = pd.read_csv("data.csv")

In [35]:
df

,INPATIENT_DATA_ID_x,DEPARTMENT_ID,DEPARTMENT_NAME,VISIT_TYPE,APPT_LENGTH,APPT_STATUS_NAME,CONTACT_DATE,APPT_DTTM,CHECKIN_DTTM,CHECKOUT_DTTM,APPT_MADE_DATE,APPT_CANC_DATE,CHAIR_START,INFUSION_START,INFUSION_END,CHAIR_OUT,ORDER_DESCRIPTION,ORDER_STATUS,CHAIR,Unnamed: 19
0,APTT001,DEPT001,INFUSION Department,INFUSION 30 MIN,30,Arrived,10/31/21,2021-10-31 02:00PM,2021-10-31 01:28PM,NaN,10/3/21,NaN,1:40 PM,NaN,NaN,NaN,NaN,*Unspecified Order Status,Chair 1,NaN
1,APTT002,DEPT001,INFUSION Department,INFUSION 2 HR,150,Arrived,10/31/21,2021-10-31 12:00PM,2021-10-31 12:21PM,NaN,10/3/21,NaN,1:15 PM,NaN,NaN,NaN,NaN,*Unspecified Order Status,Chair 2,NaN
2,APTT003,DEPT001,INFUSION Department,INFUSION 4 HR,270,Arrived,10/31/21,2021-10-31 01:00PM,2021-10-31 12:52PM,NaN,10/14/21,NaN,1:15 PM,1:15 PM,3:20 PM,NaN,ARSENIC TRIOXIDE IN D5W/NS 250 ML IVPB,Completed [5],Chair 2,NaN
3,APTT004,DEPT001,INFUSION Department,INFUSION 1 HR,90,Completed,10/31/21,2021-10-31 09:00AM,2021-10-31 08:50AM,2021-10-31 12:41PM,9/19/21,NaN,9:20 AM,11:15 AM,11:55 AM,NaN,PEMBROLIZUMAB 200 MG IN NS 59 ML,Completed [5],Chair 3,NaN
4,APTT005,DEPT001,INFUSION Department,INFUSION 2 HR,150,Canceled,10/31/21,2021-10-31 02:00PM,NaN,NaN,10/24/21,10/31/21,NaN,NaN,NaN,NaN,NaN,*Unspecified Order Status,Chair 4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,APTT419,DEPT001,INFUSION Department,INFUSION 2 HR,150,Arrived,11/4/21,2021-11-04 01:30PM,2021-11-04 02:12PM,NaN,10/5/21,NaN,2:30 PM,3:45 PM,5:00 PM,NaN,GEMCITABINE (AS 1 G/26.3 ML) IN NS INFUSION,11/4,Completed [5],Chair 9
763,APTT420,DEPT001,INFUSION Department,INFUSION 2 HR,150,Completed,11/4/21,2021-11-04 03:30PM,2021-11-04 02:17PM,2021-11-04 03:53PM,10/14/21,NaN,2:40 PM,4:00 PM,3:40 PM,NaN,VINCRISTINE IN NS INFUSION,11/4,Completed [5],Chair 10
764,APTT421,DEPT001,INFUSION Department,INFUSION 1 HR,90,Arrived,11/4/21,2021-11-04 01:30PM,2021-11-04 02:36PM,NaN,10/10/21,NaN,2:55 PM,2:15 PM,3:50 PM,NaN,ZOLEDRONIC ACID 4 MG/100ML IV SOLN PREMIX OVER...,11/4,Completed [5],Chair 24
765,APTT422,DEPT001,INFUSION Department,INFUSION 5 HR,30,Arrived,11/4/21,2021-11-04 12:00PM,2021-11-04 04:13PM,NaN,10/27/21,NaN,4:30 PM,4:45 PM,5:30 PM,NaN,SODIUM CHLORIDE 0.9 % BOLUS,11/4,Completed [5],Chair 18


In [28]:
print(df.isnull().sum())

INPATIENT_DATA_ID_x      0
DEPARTMENT_ID            0
DEPARTMENT_NAME          0
VISIT_TYPE               0
APPT_LENGTH              0
APPT_STATUS_NAME         0
CONTACT_DATE             0
APPT_DTTM                0
CHECKIN_DTTM            58
CHECKOUT_DTTM          387
APPT_MADE_DATE           0
APPT_CANC_DATE         720
CHAIR_START             59
INFUSION_START         182
INFUSION_END           196
CHAIR_OUT              767
ORDER_DESCRIPTION      182
ORDER_STATUS             0
CHAIR                    0
Unnamed: 19            608
dtype: int64


In [52]:
#this code identifies duplicates

duplicates = df.duplicated(subset='INPATIENT_DATA_ID_x', keep=False)

df = df.drop_duplicates(subset=df.columns, keep="first")

df[duplicates]



,INPATIENT_DATA_ID_x,DEPARTMENT_ID,DEPARTMENT_NAME,VISIT_TYPE,APPT_LENGTH,APPT_STATUS_NAME,CONTACT_DATE,APPT_DTTM,CHECKIN_DTTM,CHECKOUT_DTTM,APPT_MADE_DATE,APPT_CANC_DATE,CHAIR_START,INFUSION_START,INFUSION_END,CHAIR_OUT,ORDER_DESCRIPTION,ORDER_STATUS,CHAIR,Unnamed: 19
11,APTT012,DEPT001,INFUSION Department,BLOOD PRODUCT TRANSFUSION,240,Arrived,10/31/21,2021-10-31 07:00AM,2021-10-31 08:05AM,NaN,10/6/21,NaN,8:35 AM,3:15 PM,NaN,NaN,MAGNESIUM SULFATE 2 G IN NS 100 ML IVPB PREMIX,Verified [10],Chair 7,NaN
12,APTT012,DEPT001,INFUSION Department,BLOOD PRODUCT TRANSFUSION,240,Arrived,10/31/21,10/31/21 7:00,2021-10-31 08:05AM,NaN,10/6/21,NaN,8:35 AM,9:00 AM,11:00 AM,NaN,ARSENIC TRIOXIDE IN D5W/NS 250 ML IVPB,Completed [5],Chair 7,NaN
14,APTT014,DEPT001,INFUSION Department,INFUSION 3 HR,180,Completed,10/31/21,2021-10-31 08:00AM,2021-10-31 08:35AM,2021-10-31 01:52PM,10/18/21,NaN,10:30 AM,10:15 AM,12:20 PM,NaN,BLEOMYCIN SULFATE 15 UNITS IJ SOLR,Completed [5],Chair 6,NaN
15,APTT014,DEPT001,INFUSION Department,INFUSION 3 HR,180,Completed,10/31/21,2021-10-31 08:00AM,2021-10-31 08:35AM,2021-10-31 01:52PM,10/18/21,NaN,10:30 AM,10:15 AM,12:50 PM,NaN,DACARBAZINE IN NS/D5W INFUSION,Completed [5],Chair 6,NaN
16,APTT014,DEPT001,INFUSION Department,INFUSION 3 HR,180,Completed,10/31/21,2021-10-31 08:00AM,2021-10-31 08:35AM,2021-10-31 01:52PM,10/18/21,NaN,10:30 AM,10:15 AM,10:07 AM,NaN,FAMOTIDINE 20 MG/2ML IV SOLN,Completed [5],Chair 6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,APTT407,DEPT001,INFUSION Department,INFUSION 2 HR,120,Arrived,11/4/21,2021-11-04 12:30PM,2021-11-04 12:19PM,NaN,10/31/21,NaN,12:50 PM,2:30 PM,3:07 PM,NaN,SODIUM CHLORIDE 0.9 % FOR IN OFFICE HYDRATION,11/4,Completed [5],Chair 24
754,APTT413,DEPT001,INFUSION Department,INFUSION 2 HR,150,Completed,11/4/21,2021-11-04 10:00AM,2021-11-04 01:19PM,2021-11-04 04:27PM,11/4/21,NaN,1:30 PM,1:45 PM,1:45 PM,NaN,SODIUM CHLORIDE 0.9 % BOLUS,11/4,Completed [5],Chair 3
755,APTT413,DEPT001,INFUSION Department,INFUSION 2 HR,150,Completed,11/4/21,2021-11-04 10:00AM,2021-11-04 01:19PM,2021-11-04 04:27PM,11/4/21,NaN,1:30 PM,1:11 PM,NaN,NaN,ONDANSETRON HCL 4 MG/2ML IJ SOLN,11/4,Verified [10],Chair 3
759,APTT417,DEPT001,INFUSION Department,INFUSION 4 HR,270,Arrived,11/4/21,11/4/21 15:00,2021-11-04 02:10PM,NaN,11/4/21,NaN,4:00 PM,2:45 PM,7:30 PM,NaN,MAGNESIUM SULFATE IN NS/D5W 100 ML IVPB,11/4,Completed [5],Chair 22


In [58]:
df.drop_duplicates(subset='INPATIENT_DATA_ID_x', keep='first', inplace=True)

df

df.to_csv('nonDuplicateData.csv', index=False)

In [25]:
df.drop(columns=['INPATIENT_DATA_ID_x'], inplace=True)
df.drop(columns=['DEPARTMENT_ID'], inplace=True)
df.drop(columns=['DEPARTMENT_NAME'], inplace=True)
